In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from ipython_helpers import (
    print_full
)
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
import readline
from IPython.display import display

In [2]:
GLOBAL_INPUT_VARIABLES = [
    'CASHMTA_win',
    'EXRET_AVG_win',
    'MB_win',
    'NIMTA_AVG_win',
    'RSIZE_win', 
    'PRICE_win',
    'SIGMA_win', 
    'TLMTA_win'    
]

def transform_data(df):
    if 'Unnamed: 0' in df:
        del df['Unnamed: 0']
    
    return df

    
def get_num_of_bankruptcy_df(df):
    df = df.drop_duplicates(subset=['PERMNO'], keep='first')
    bankruptcy_df = df[df['is_bankrupt'] == True]
    num_of_bankruptcy_series = bankruptcy_df.groupby(by=['deletion_year']).count()['is_bankrupt']
    num_of_bankruptcy_df = pd.DataFrame(num_of_bankruptcy_series)
    num_of_bankruptcy_df.rename(columns={'is_bankrupt': 'Bankruptcies'}, inplace=True)
    num_of_bankruptcy_df.index = num_of_bankruptcy_df.index.astype(np.int32)

    return num_of_bankruptcy_df
    
    
def get_num_of_active_firms_df(df):  
    df = df.drop_duplicates(subset=['PERMNO', 'year'], keep='first')
    num_of_active_firms_series = df.groupby(by=['year']).count()['PERMNO']
    num_of_active_firms_df = pd.DataFrame(num_of_active_firms_series)
    num_of_active_firms_df.rename(columns={'PERMNO': 'Active Firms'}, inplace=True)
    
    return num_of_active_firms_df


def get_firm_status_by_year(df):
    num_of_active_firms_df = get_num_of_active_firms_df(df)
    num_of_bankruptcy_df = get_num_of_bankruptcy_df(df)

    df = num_of_active_firms_df.merge(num_of_bankruptcy_df, how='left', left_index=True, right_index=True)
    df.reset_index(inplace=True)
    df.rename(
        columns={
            'year': 'Year',
        },
        inplace=True
    )
    df.reset_index(inplace=True)
    df['(%)'] =(df['Bankruptcies'] / df['Active Firms'] * 100).round(4)
    df = df[['Year', 'Active Firms', 'Bankruptcies', '(%)']]

    return df


def get_observations_by_columns(df):
    input_df = df[['PERMNO', 'NIMTA_AVG_win', 'TLMTA_win', 'CASHMTA_win', 'RSIZE_win', 'PRICE_win', 'EXRET_AVG_win', 'SIGMA_win', 'MB_win']]
    
    observations_by_columns_df = pd.DataFrame(input_df.count(axis=0), columns=['Num of Observatons'])
                                    
    return observations_by_columns_df    


def get_observations_by_num_of_vars(df):
    input_df = df[['PERMNO', 'NIMTA_AVG_win', 'TLMTA_win', 'CASHMTA_win', 'RSIZE_win', 'PRICE_win', 'EXRET_AVG_win', 'SIGMA_win', 'MB_win']]

    num_of_vars_df = pd.DataFrame(pd.DataFrame(input_df.count(axis=1), columns=['Num of Observations']))
    observations_by_num_of_vars_df = pd.DataFrame(num_of_vars_df['Num of Observations'].value_counts(), columns=['Num of Observations'])
    
    return observations_by_num_of_vars_df.sort_index(ascending=False)
        

def compute_months_diff(row):
    year_diff = row['deletion_year'] - row['year']
    months_diff = row['deletion_month'] - row['month']
    
    if year_diff > 0:
        return 12 * year_diff + months_diff 
    else:
        return months_diff 

    
def make_months_diff_df(df, find_closest=False):
    bankruptcy_df = df[df['is_bankrupt'] == True]
    bankruptcy_df = bankruptcy_df.sort_values(by=['PERMNO', 'year', 'month'])
    if find_closest:
        bankruptcy_df.drop_duplicates(subset=['PERMNO'], keep='last', inplace=True)

    bankruptcy_df.loc[:, 'months_diff'] = bankruptcy_df.apply(lambda x: compute_months_diff(x), axis=1)
    
    return bankruptcy_df 


def filter_months_diff_df(df, threshold_diff):

    closest_months_diff_df = make_months_diff_df(df, find_closest=True)
    large_months_diff_df = closest_months_diff_df[closest_months_diff_df['months_diff'] > threshold_diff]
    security_filter_1 = large_months_diff_df['PERMNO'].values

    months_diff_df = make_months_diff_df(df)
    small_months_diff_df = months_diff_df[months_diff_df['months_diff'] < threshold_diff]
    index_filter = small_months_diff_df.index 
    
    output_df = df[~df['PERMNO'].isin(security_filter_1)]
    output_df = output_df.drop(index_filter) 
    
    bankruptcy_list_all = output_df[output_df['is_bankrupt'] == True]['PERMNO'].unique()
    bankruptcy_list_specific = output_df[output_df['is_bankrupt_12'] == True]['PERMNO'].unique()
    diff_list = list(set(bankruptcy_list_all).symmetric_difference(set(bankruptcy_list_specific)))
    
    output_df = output_df[~output_df['PERMNO'].isin(diff_list)]
    
    return output_df
        
    
def plot_histogram_of_diff_in_dates(df):
    max_num = df['months_diff'].max() + 1
    df['months_diff'].hist(bins=max_num + 1, range=(0, max_num))

    return 
    
def get_complete_features(df):
    df = df.dropna(how='any', subset=GLOBAL_INPUT_VARIABLES)
    
    return df


def filter_year_df(df, start_year=1970, end_year=2015):
    return df[(df['year'] >= start_year) & (df['year'] <= end_year)]


def make_train_and_test_df(df, threshold_year=2005):
    train_df = df[df['year'] <= threshold_year]
    test_df = df[df['year'] > threshold_year]
    
    return train_df, test_df


def get_num_of_bankruptcies(df):
    return len(df[df['is_bankrupt'] == True]['PERMNO'].unique())


def remove_bankrupt_observations_with_missing_12_months(df):
    """Remove observations where "is_bankrupt" is marked as True
    but "is_bankrupt_within_12" is marked as False.
    This removal is necessary because while the bankrupt observations have
    features that are indicative of bankrupcy, the False mark for "is_bankrupt_within_12"
    would confuse the model.
    """
    bankrupty_df = df[df['is_bankrupt'] == True]
    all_list = bankruptcy_df['PERMNO'].unique()
    available_list = bankruptcy_df[bankruptcy_df['is_bankrupt_within_12']]['PERMNO'].unique()
    missing_list = list(set(all_list).symmetric_difference(set(available_list)))
    
    return df[~df['PERMNO'].isin(missing_list)]

In [3]:
original_df = pd.read_csv("../../../../Development/econ985/campbell_data/simple_quarterly_campbell_df_6.csv", parse_dates=['datadate', 'lagged_date', 'dldte'])

In [4]:
transformed_df = transform_data(original_df)

In [5]:
transformed_df.shape

(808988, 20)

In [6]:
get_firm_status_by_year(transformed_df)

,Year,Active Firms,Bankruptcies,(%)
0,1961,40,NaN,NaN
1,1962,502,NaN,NaN
2,1963,549,NaN,NaN
3,1964,595,NaN,NaN
4,1965,653,NaN,NaN
5,1966,1391,NaN,NaN
6,1967,1493,NaN,NaN
7,1968,1652,NaN,NaN
8,1969,1807,NaN,NaN
9,1970,1922,1.0,0.0520


In [7]:
filtered_year_df = filter_year_df(transformed_df, start_year=1970, end_year=2015)
get_firm_status_by_year(filtered_year_df)

,Year,Active Firms,Bankruptcies,(%)
0,1970,1922,1,0.0520
1,1971,2017,5,0.2479
2,1972,2118,3,0.1416
3,1973,2476,7,0.2827
4,1974,2490,4,0.1606
5,1975,2493,6,0.2407
6,1976,2477,12,0.4845
7,1977,2450,6,0.2449
8,1978,2394,13,0.5430
9,1979,2339,14,0.5985


In [8]:
complete_df = get_complete_features(filtered_year_df)

In [9]:
print(filtered_year_df.shape)
print(get_num_of_bankruptcies(filtered_year_df))
print(complete_df.shape)
print(get_num_of_bankruptcies(complete_df))


(761876, 20)
1226
(680042, 20)
1194


In [10]:
get_firm_status_by_year(complete_df)

,Year,Active Firms,Bankruptcies,(%)
0,1970,1,NaN,NaN
1,1971,414,NaN,NaN
2,1972,1767,NaN,NaN
3,1973,2010,NaN,NaN
4,1974,2203,NaN,NaN
5,1975,2259,NaN,NaN
6,1976,2290,9.0,0.3930
7,1977,2414,6.0,0.2486
8,1978,2359,13.0,0.5511
9,1979,2289,14.0,0.6116


In [11]:
filtered_year_df1 = filter_year_df(complete_df, start_year=1976, end_year=2015)

In [12]:
get_firm_status_by_year(filtered_year_df1)

,Year,Active Firms,Bankruptcies,(%)
0,1976,2290,2,0.0873
1,1977,2414,6,0.2486
2,1978,2359,12,0.5087
3,1979,2289,13,0.5679
4,1980,2228,13,0.5835
5,1981,2409,7,0.2906
6,1982,4002,12,0.2999
7,1983,4370,13,0.2975
8,1984,4758,13,0.2732
9,1985,4755,24,0.5047


In [13]:
true_bankruptcy_df = filtered_year_df1[filtered_year_df1['is_bankrupt_within_6'] == True]
print(len(true_bankruptcy_df['PERMNO'].unique()))

154


In [14]:
filtered_year_df1.shape

(668382, 20)

In [15]:
train_df, test_df = make_train_and_test_df(filtered_year_df1)

# Save Data

In [16]:
train_df.to_csv("../../../../Development/econ985/campbell_data/train_df_quarterly_6.csv")
test_df.to_csv("../../../../Development/econ985/campbell_data/test_df_quarterly_6.csv")